# Awaybot
Kevin Davis, Walter Erquinigo, Guillermo Monge, Carlos Rodriguez & Alex Smith

***
### Resources
- <a href="https://www.fullstackpython.com/blog/build-first-slack-bot-python.html">How to Build Your First Slack Bot with Python</a>
- <a href="https://medium.com/@julianmartinez/how-to-write-a-slack-bot-with-python-code-examples-4ed354407b98#.t8q6p76kj">How To Write a Slack Bot — with Python Code Examples</a>
- <a href="http://angus.readthedocs.io/en/2014/amazon/transfer-files-between-instance.html">Transfer Files Between Your Laptop and an EC2 Instance</a>

***
Add the file from your local computer to the EC2 instance:<br>
```
scp -i /Users/Alex/Documents/Berkeley/1603Fall/W210/Architecture/Capstone.pem /Users/Alex/Documents/Berkeley/1603Fall/W210/Slackbot/awaybot.py root@ec2-54-175-51-154.compute-1.amazonaws.com:/usr/lib/
```

***
### Test out the bot to ensure it's working all good

In [1]:
#######################################
#
# The below need to be populted for bot
# to function
#
#######################################
%env SLACK_BOT_TOKEN=#SLACK_BOT_TOCKEN#
%env BOT_NAME=#SLACK_BOT_NAME#
%env IMAGE_URL=https://raw.githubusercontent.com/WillahScott/slack-pack/dev/data/grumpy_cat_2.png

In [19]:
%%writefile print_bot_id.py
import os
from slackclient import SlackClient


BOT_NAME = os.environ.get('BOT_NAME')

slack_client = SlackClient(os.environ.get('SLACK_BOT_TOKEN'))

if __name__ == "__main__":
    api_call = slack_client.api_call("users.list")
    if api_call.get('ok'):
        # retrieve all users so we can find our bot
        users = api_call.get('members')
        for user in users:
            if 'name' in user and user.get('name') == BOT_NAME:
                print("Bot ID for '" + user['name'] + "' is " + user.get('id'))
        print slack_client.api_call("channels.list")
    else:
        print("could not find bot user with the name " + BOT_NAME)

Overwriting print_bot_id.py


In [2]:
!python print_bot_id.py

In [13]:
# set the bot_id as a global environment variable
!export BOT_ID='U2JFJ5ETU'

### Create a sample simpleDB with grumpy cat

In [3]:
import boto3

# connect to simpleDB
simple_client = boto3.client('sdb')

# create a test domain (think of a domain as a table)
simple_client.create_domain(DomainName='test')

# create a entry in this table, this lone entry
# is the link to a grumpy cat picture
test_item = 'grumpy'
test_attr = [{"Name":"Link",
              "Value":'https://raw.githubusercontent.com/WillahScott/slack-pack/dev/data/grumpy_cat_2.png'}]

# load the entry into simpleDB
simple_client.put_attributes(DomainName='test',
                             ItemName=test_item,
                             Attributes=test_attr)

{'ResponseMetadata': {'BoxUsage': '0.0000219909',
  'HTTPHeaders': {'connection': 'keep-alive',
   'content-type': 'text/xml',
   'date': 'Fri, 11 Nov 2016 15:03:01 GMT',
   'server': 'Amazon SimpleDB',
   'transfer-encoding': 'chunked',
   'vary': 'Accept-Encoding'},
  'HTTPStatusCode': 200,
  'RequestId': '74e765a9-d8ca-2b8a-c47f-e0da4486f93d',
  'RetryAttempts': 0}}

In [25]:
# read back grumpy cat from the database
grumpy = simple_client.get_attributes(DomainName="test",ItemName="grumpy")

# print out the whole record
print grumpy
print "\n"

# print out just the link
print grumpy['Attributes'][0]['Value']

{u'Attributes': [{u'Name': 'Link', u'Value': 'https://raw.githubusercontent.com/WillahScott/slack-pack/dev/data/grumpy_cat_2.png'}], 'ResponseMetadata': {'HTTPStatusCode': 200, 'BoxUsage': '0.0000093222', 'RequestId': 'fdc77cd6-b023-f711-7b39-93fbcc6d6170'}}


https://raw.githubusercontent.com/WillahScott/slack-pack/dev/data/grumpy_cat_2.png


### Awaybot code

In [68]:
!ls

awaybot.ipynb  awaybot.py  exceptions.py  grump.png  log  print_bot_id.py


In [1]:
%%writefile awaybot.py
import os
import time
from slackclient import SlackClient
import boto3
import logging
from logging.handlers import RotatingFileHandler

# create logger
logger = logging.getLogger('awaybot_logger')
logger.setLevel(logging.DEBUG)
LOGFILE = "log/awaybot"

# create formatter
formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')

# create console handler, set level of logging and add formatter
ch = logging.StreamHandler()
ch.setLevel(logging.DEBUG)
ch.setFormatter(formatter)

# create file handler, set level of logging and add formatter
fh = logging.handlers.TimedRotatingFileHandler(LOGFILE, when='M', interval =1)
fh.setLevel(logging.DEBUG)
fh.setFormatter(formatter)
fh.suffix = '%Y-%m-%d_%H-%M-%S.log'

# add handlers to logger
logger.addHandler(ch)
logger.addHandler(fh)

class Slackbot:
    """
    A class for our slackbot
    """
    def __init__(self,
                 slack_client,
                 bot_name,
                 command):
        """
        Constructor for Slackbot class
        
        Parameters
        ----------
        slack_client : object
            The slack client object to interact with a slack team server
        bot_name : str
            The name (not ID) of the slackbot for the relevant slack team
        command : str
            The command to invoke the slackbot
        """
        try:
            logger.info("""Initializing bot with parameters:
            \tslack_client : %s
            \tbot_name : %s
            \tcommand : %s""" % (slack_client, bot_name, command))
            self.slack_client = slack_client
            self.command = command
            self.bot_name = bot_name
            self.sdb_status = False
            api_call = self.slack_client.api_call("users.list")
            if api_call.get('ok'):
                # retrieve all users so we can find our bot
                users = api_call.get('members')
                # indicator for whether a user has been found matching our bot
                user_found = False
                for user in users:
                    if 'name' in user and user.get('name') == self.bot_name:
                        self.bot_id = user.get('id')
                        user_found = True
                        print("Bot ID for '" + user['name'] + "' is " + user.get('id'))
                if not user_found:
                    raise SlackUserNotFoundError("Could not find bot user with the name " + self.bot_name)
        except SlackUserNotFoundError as e:
            print 'SlackUserNotFoundError:', e.value
            # at this point, quitting if we hit this issue
            quit()
    
    def simpledbConnect(self):
        """
        Function that uses the supplied token to connect to the AWS simpleDB
        client. Quits if failure to connect.

        Parameters:
        -----------
        None

        Returns: 
        ----------
        None
        """
        try:
            self.sdb = boto3.client('sdb')
        except:
            logger.error(
                "Failed to connect to AWS. Have you configured "
                "AWS CLI?", exc_info=True)
            sys.exit()
        else:
            self.sdb_status = True
        return

    def handle_command(self, team, command, simpleDB_domain):
        """
        Receives commands directed at the bot and determines if they
        are valid commands. If so, then acts on the commands. If not,
        returns back what it needs for clarification.
        
        Parameters
        ----------
        command : dtr
            the command passed to the slackbot
        payload : str
            the location of the payload to be delivered by slackbot 
        """
        # split up the command into its component parts
        if not self.sdb_status:
            self.simpledbConnect()
        split_command = command.split()
        
        # ensure that command sent to bot matches
        # and is well formatted
        if command.startswith(self.command) and len(split_command) == 4:

            # command components
            command_channel = split_command[1]
            command_duration = split_command[2]
            command_duration_units = split_command[3]
            channel_name = command_channel.split('|')[-1][:-1]
            sdbItem = '{}_{}_{}_{}'.format(
                team_name, channel_name,
                command_duration, command_duration_units)
            logger.info('Fetching number of items for {}'.format(sdbItem))
            fetch_num_topics = self.sdb.get_attributes(
                DomainName='awaybot', ItemName=sdbItem, ConsistentRead=True)

            ## CASE ONE: VALID COMMAND BUT INVALID TIME RANGE
            try:
                num_topics = fetch_num_topics['Attributes'][0]['Value']
            except:
                logger.info("Could not fetch number of topics from Simple DB for COMMAND '%s'" % (command))
                response = ("""Not sure what you mean. Use the */summarize* command with the *channel name* and the *duration*. 
                    For example, if you want to see 3 weeks of history in the #general channel, type: */summarize #general 3 weeks*.
                    Valid time values are 1-12 hours, 1-6 days, 1-6 weeks.""")

                # post the response to the channel
                self.slack_client.api_call("chat.postMessage",
                                           channel=channel,
                                           text=response,
                                           as_user=True)
            else:
                logger.info("""COMMAND %s has a valid format for processing:
                command_channel : %s
                command_duration : %s
                command_duration_units : %s""" % (command,
                                                  command_channel,
                                                  command_duration,
                                                  command_duration_units))

                ## CASE TWO: VALID COMMAND BUT NOT ENOUGH MESSAGES FOR WORDCLOUD GENERATION
                if not int(num_topics):
                    response = "{} has no topics for {} {}".format(
                        command_channel, command_duration, command_duration_units)
                    self.slack_client.api_call("chat.postMessage",
                                           channel=channel,
                                           text=response,
                                           as_user=True)
                else:
                    ## CASE THREE: VALID COMMAND AND WORDCLOUDS WERE GENERATED.
                    
                    # Respond with the summary and how many topics
                    response = "Your summary for {} for {} {} ({} topics).".format(
                        command_channel, command_duration, command_duration_units, num_topics)

                    self.slack_client.api_call("chat.postMessage",
                                           channel=channel,
                                           text=response,
                                           as_user=True)


                    # Respond for each topic
                    for topic in xrange(1, int(num_topics), 1):
                        sdbImageItem = '{}_{}_{}_{}_{}'.format(
                            team_name, channel_name,
                            command_duration, command_duration_units, topic)

                        fetch_image = self.sdb.get_attributes(
                            DomainName='awaybot', ItemName=sdbImageItem, ConsistentRead=True)
                        topic_url = [i['Value'] for i in fetch_image['Attributes'] if i['Name'] == 'archiveURL'][0]
                        topic_wordcloud = [i['Value'] for i in fetch_image['Attributes'] if i['Name'] == 'modelURL'][0]


                        response = '<{}|Go To Topic {}>'.format(topic_url, topic)
                        # set the attachment which is the duration with the url to the word cloud
                        attachment = '[{"title": "Topic '+ str(topic) + '" , "image_url":"'+ topic_wordcloud +'"}]'

                        # send the response with the image
                        logger.info("""Sending:
                        \tchannel : %s
                        \ttext : %s
                        \timage_url : %s""" % (channel, response, attachment))
                        self.slack_client.api_call("chat.postMessage", channel=channel, 
                                              text=response, 
                                              attachments=attachment,
                                              as_user=True)

        # otherwise post our response to help the user our
        else:
            # response for when the bot doesn't know quite what to do
            logger.info("Incorrectly formatted COMMAND '%s' sent to BOT '%s'" % (command, self.bot_name))
            response = ("""Not sure what you mean. Use the */summarize* command with the *channel name* and the *duration*. 
                For example, if you want to see 3 weeks of history in the #general channel, type: */summarize #general 3 weeks*.
                Valid time values are 1-12 hours, 1-6 days, 1-6 weeks.""")

            # post the response to the channel
            self.slack_client.api_call("chat.postMessage",
                                       channel=channel,
                                       text=response,
                                       as_user=True)
            
    def parse_messages(self,
                       slack_rtm_output):
        """
        The Slack Real Time Messaging API is an events firehose.
        this parsing function returns None unless a message is
        directed at the Bot, based on its ID.        
        Parameters
        ----------
        slack_rtm_output : list
            A list of outputs from the slack RTM API
        Returns
        -------
        str
            str slice after @ for identified slack RTM API text message
        str
            slack team channel ID
        """
        # by default set the output to what was spit out 
        # by the fire hose
        output_list = slack_rtm_output
        # if we actually got something and not just an
        # empty array
        if output_list and len(output_list) > 0:

            # loop through each of the messages
            for output in output_list:

                # if we have some text in the message and 
                # this bot is mentioned
                if output and 'text' in output and self.bot_id in output['text']:

                    # return text after the @ mention, whitespace removed
                    logger.info("BOT '%s' has received MESSAGE '%s'" % (self.bot_name, output))
                    at_bot = "<@" + self.bot_id + ">"
                    return output['text'].split(at_bot)[1].strip().lower(), \
                           output['channel']
        return None, None

class SlackUserNotFoundError(Exception):
    """
    Exception raised for error encountered for inexistent slack user name
    
    Attributes:
        message -- explanation of the error
    """
    def __init__(self, value):
        self.value = value

    def __str__(self):
        return repr(self.value)
        
class UnauthError(Exception):
    """
    Exception raised for error encountered for invalid slack credentials/token
    
    Attributes:
        message -- explanation of the error
    """
    def __init__(self, value):
        self.value = value

    def __str__(self):
        return repr(self.value)

if __name__ == "__main__":
    
    try:
        #######################################
        # THIS IS PREDEFINED PAYLOAD CODE START
        #######################################
        
        logger.info("Stating __main__, simple test for awaybot.py")
        
        SDB_DOMAIN = "awaybot"
        # SDB_ITEM = "test"
        
        # # connect to simpleDB
        # logger.info("Connecting to simpleDB")
        # simple_client = boto3.client('sdb')

        # # pull the relevant simpleDB entry
        # logger.info("Pulling ITEM '%s' from DOMAIN '%s'" % (SDB_ITEM, SDB_DOMAIN))
        # response = simple_client.get_attributes(DomainName=SDB_DOMAIN,
        #                                       ItemName=SDB_ITEM,
        #                                       ConsistentRead=True)

        # # gather just the link to the word cloud
        # word_cloud = [i['Value'] for i in response['Attributes'] if i['Name'] == 'modelURL'][0]
        # topic_url = [i['Value'] for i in response['Attributes'] if i['Name'] == 'archiveURL'][0]
        # payload = [word_cloud, topic_url]
        # logger.info("summary wordcloud URL: {}\ntopic url: {}".format(word_cloud, topic_url))

        #######################################
        # THIS IS PREDEFINED PAYLOAD CODE END
        #######################################

        READ_WEBSOCKET_DELAY = 1
        logger.info("RTM API read websocket delay set to '%s' seconds" % (READ_WEBSOCKET_DELAY))

        # try to connect to the real time messaging API
        sc = SlackClient(os.environ.get('SLACK_BOT_TOKEN'))

        bot = Slackbot(sc,
                     os.environ.get('SLACK_BOT_NAME'),
                     '/summarize')
        bot.simpledbConnect()

        team_name = sc.api_call('team.info')['team']['name']

        if bot.slack_client.rtm_connect():
            logger.info("starterbot connected and running!")
            # keep a continuous loop
            while True:
                # parse the output from the client using our
                # pre-defined parser function
                command, channel = bot.parse_messages(bot.slack_client.rtm_read())

                # if we have both a command and a channel, then
                # use our predefined handle command function
                # NOTE THAT THE PAYLOAD IS PREDEFINED
                if command and channel:
                    bot.handle_command(team_name, command, SDB_DOMAIN)

                # sleep for the number of seconds before reading from
                # the fire hose again
                time.sleep(READ_WEBSOCKET_DELAY)

        # if something went wrong, print out what happened
        else:
            raise UnauthError("Credentials/token invalid")
    except UnauthError as e:
        print 'UnauthError:', e
        quit()

Writing awaybot.py


In [70]:
# start up the away bot
!python awaybot.py

2016-11-12 21:57:09,128 - INFO - Stating __main__, simple test for awaybot.py
2016-11-12 21:57:09,128 - INFO - Connecting to simpleDB
2016-11-12 21:57:09,226 - INFO - Pulling ITEM 'grumpy' from DOMAIN 'test'
2016-11-12 21:57:09,287 - INFO - summary payload URL: 'https://raw.githubusercontent.com/WillahScott/slack-pack/dev/data/grumpy_cat_2.png'
2016-11-12 21:57:09,287 - INFO - RTM API read websocket delay set to '1' seconds
2016-11-12 21:57:09,287 - INFO - Initializing bot with parameters:
            	slack_client : <slackclient._client.SlackClient object at 0x7f01ad21f0d0>
            	bot_name : starterbot
            	command : /summarize
Bot ID for 'starterbot' is U2CN6DVUZ
2016-11-12 21:57:09,530 - INFO - starterbot connected and running!
^C
Traceback (most recent call last):
  File "awaybot.py", line 264, in <module>
    time.sleep(READ_WEBSOCKET_DELAY)
KeyboardInterrupt


### Testing the Slackbot with just sending messages

#### Choose a channel

In [54]:
from slackclient import SlackClient
import os

# initalize the client
slack_client = SlackClient('TOKEN')

# get all the channels
all_channels = slack_client.api_call("channels.list")['channels']

# set the channel of interest
channel_interest_name = 'bot-sandbox'
channel_interest = None

# loop through all the channels, looking for the channel of
# interest
for channel in all_channels:
    if channel['name'] == channel_interest_name:
        channel_interest = channel['id']
        print channel_interest

C2CNEMD0S


#### Post a simple message

In [69]:
from slackclient import SlackClient

# set the response that we'd like to send to the channel
response = "Hmmm..."

# instantiate Slack client as the slackbot
slack_client = SlackClient('TOKEN')

# send a text message
slack_client.api_call("chat.postMessage", channel='C2CNEMD0S',
                      text=response, as_user=True)

{u'channel': u'C2CNEMD0S',
 u'message': {u'bot_id': u'B2JDJJUUB',
  u'text': u'Hmmm...',
  u'ts': u'1478571838.000066',
  u'type': u'message',
  u'user': u'U2JFJ5ETU'},
 u'ok': True,
 u'ts': u'1478571838.000066'}

#### Upload a file

In [60]:
from slackclient import SlackClient

# set the response that we'd like to send to the channel
response = "/Users/Alex/Documents/Berkeley/1603Fall/W210/Slackbot/grumpy_cat_2.png"

# instantiate Slack client as the slackbot
slack_client = SlackClient('TOKEN')

# upload a picture as a message
slack_client.api_call("files.upload", channel='C2CNEMD0S',
                          file=response, title="grumpy_cat_2.png",filetype='image')

{u'file': {u'channels': [],
  u'comments_count': 0,
  u'created': 1478570603,
  u'display_as_bot': False,
  u'edit_link': u'https://slackpack-project.slack.com/files/awaybot_test_aks/F2ZLNC4UD/file/edit',
  u'editable': True,
  u'external_type': u'',
  u'filetype': u'text',
  u'groups': [],
  u'id': u'F2ZLNC4UD',
  u'ims': [],
  u'is_external': False,
  u'is_public': False,
  u'lines': 1,
  u'lines_more': 0,
  u'mimetype': u'text/plain',
  u'mode': u'snippet',
  u'name': u'file',
  u'permalink': u'https://slackpack-project.slack.com/files/awaybot_test_aks/F2ZLNC4UD/file',
  u'permalink_public': u'https://slack-files.com/T2BT8MVE3-F2ZLNC4UD-293b7f70aa',
  u'pretty_type': u'Plain Text',
  u'preview': u'/Users/Alex/Documents/Berkeley/1603Fall/W210/Slackbot/grumpy_cat_2.png',
  u'preview_highlight': u'<div class="CodeMirror cm-s-default CodeMirrorServer" oncopy="if(event.clipboardData){event.clipboardData.setData(\'text/plain\',window.getSelection().toString().replace(/\\u200b/g,\'\'));eve